In [2]:
import numpy as np
from astropy.table import Table
import FoFCatalogMatching

n = 100
ra = np.random.uniform(0, 360, n)
dec = np.random.uniform(-90, 90, n)

cat_a = Table({'ra': ra, 'dec': dec})
cat_b = Table({'ra': ra+np.random.normal(0, 0.0002, n), 'dec': dec+np.random.normal(0, 0.0002, n)})

results = FoFCatalogMatching.match({'a': cat_a, 'b':cat_b}, 
                                   {3.0: 3, 2.0: 3, 1: None})

In [3]:
results

row_index,catalog_key,group_id
int64,str1,int64
0,a,0
0,b,0
1,a,1
1,b,1
2,a,2
2,b,2
3,a,3
3,b,3
4,a,4
